In [1]:
import numpy as np
import pandas as pd

def preprocess(df):
    n = df.shape[0]
    flags = [0 for i in range(n)]
    emojis = [{} for i in range(n)]
    df['review'] = df['review'].apply(lambda x : x.split("//")[0] if "//" in x else x)
    for i in range(n):
        start = []
        s = df.iloc[i, 1]
        for j in range(len(s)):
            if s[j] == "[":
                start.append(j)
            if s[j] == "]" and len(start) > 0:
                flags[i] = 1
                emoji = s[start[-1]:(j + 1)]
                if emoji not in emojis[i]:
                    emojis[i][emoji] = 0
                emojis[i][emoji] += 1
                start.pop()
    df['has_emoji'] = flags
    df['emojis'] = emojis

In [2]:
df1 = pd.read_csv("Data/weibo_senti_100k.csv")
preprocess(df1)
df1 = df1[df1['has_emoji'] == 1][['label', 'review', 'emojis']]
print(df1.shape[0])
df1.head()

95461


,label,review,emojis
0,1,﻿更博了，爆照了，帅的呀，就是越来越爱你！生快傻缺[爱你][爱你][爱你],{'[爱你]': 3}
1,1,@张晓鹏jonathan 土耳其的事要认真对待[哈哈]，否则直接开除。@丁丁看世界 很是细心...,{'[哈哈]': 1}
3,1,美~~~~~[爱你],{'[爱你]': 1}
4,1,梦想有多大，舞台就有多大![鼓掌],{'[鼓掌]': 1}
5,1,[花心][鼓掌],"{'[花心]': 1, '[鼓掌]': 1}"


In [ ]:
df2 = pd.read_csv('Data/dianping/ratings.csv')
df2 = df2.dropna()
df2['label'] = np.where(df2['rating'] > 3, 1, 0)
df2['length'] = df2['comment'].str.len()
df2 = df2[df2['length'] <= 512]
df2 = df2[['label', 'comment']].sample(n = df1.shape[0], replace = False, random_state = 0)
df2.columns = ['label', 'review']
print(df2.shape[0])
df2.head()

In [ ]:
df1.to_csv('Data/processed_Weibo_data.csv')
df2.to_csv('Data/processed_Dianping_data.csv')